# This notebook focuses on the Installation of Trapheus along with its prerequisites section. Executing this notebook will create the resources if it doesn't exist.

Below are the parameters needed for the Trapheus installation

* s3-bucket : [Optional] The name of the CloudFormation template S3 bucket from the Pre-Requisites.
* vpcID : [Required] The id of the VPC from the Pre-Requisites. The lambdas from the Trapheus state machine will be created in this VPC.
* Subnets : [Required] A comma separated list of private subnet ids (region specific) from the Pre-Requisites VPC.
* SenderEmail : [Required] The SES sending email configured in the Pre-Requisites
* RecipientEmail : [Required] Comma separated list of recipient email addresses configured in Pre-Requisites.
* UseVPCAndSubnets : [Optional] Whether to use the vpc and subnets to create a security group and link the security group and vpc to the lambdas. When UseVPCAndSubnets left out (default) or set to 'true', lambdas are connected to a VPC in your account, and by default the function can't access the RDS (or other services) if VPC doesn't provide access (either by routing outbound traffic to a NAT gateway in a public subnet, or having a VPC endpoint, both of which incur cost or require more setup). If set to 'false', the lambdas will run in a default Lambda owned VPC that has access to RDS (and other AWS services).
* SlackWebhookUrls : [Optional] Comma separated list of Slack webhooks for failure alerts.


> Note:  Make sure to have AWS credentials configured. Refer to [Setting up AWS credentials](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html)

In [ ]:
region=input("Enter the region [for instance, us-west-2]: ")
if region == "":
    raise SystemExit("Region is empty. Provide the region to create the resources")
s3_bucket_name=input("Enter the name of the S3 bucket. New bucket will be created if it doesn't exist(Proposed Name: trapheus-cfn-s3-[account-id]-[region]): ")
isVpc = input('Do you want to use vpc[y/n]: ')
if isVpc == 'y' or isVpc == 'Y':
    vpc_id=input("Enter the VPC ID. Leaving it empty will create a new VPC along with subnets: ")
    if len(vpc_id) > 0:
        subnets = input('Enter comma separated list of PRIVATE subnets: ')
sender_email_id=input("Enter sender email to send email FROM in case of failure: ")
receiver_email_id=input("Enter recipient email to send email TO in case of failure:  ")
create_rds=input("Do you want to create an RDS instance[y/n]")
if create_rds == 'y' or create_rds == 'Y' :
    db_instance_name=input("Enter a RDS instance name")
slack_webhook_urls = input('Enter slack webhooks to publish failure notifications to: ')
stack_name = input('Enter a cloud formation stack name: ')

In [ ]:
import boto3

In [ ]:
# Verify email
ses = boto3.client('ses')
if len(sender_email_id) > 0:
    response = ses.verify_email_identity(
      EmailAddress = sender_email_id
    )
else:
    print("Sender email id not provided. Skipping this step")

if len(receiver_email_id) > 0:
    response = ses.verify_email_identity(
      EmailAddress = receiver_email_id
    )
else:
    print("Receiver email id not provided. Skipping this step")

In [ ]:
# Validate and Create S3 bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket(s3_bucket_name)

if bucket.creation_date:
   print("The bucket already exists. Proceeding to next step.")
else:
   print("The bucket does not exist. Creating new one with the given name")
   s3Response = s3.create_bucket(Bucket=s3_bucket_name,CreateBucketConfiguration={
    'LocationConstraint': region})
   print("created bucket {}".format(s3_bucket_name))
   

In [ ]:
# If VPC id is not provided, then create VPC along with private subnets
ec2 = boto3.resource('ec2')
ec2Client = boto3.client('ec2')
if vpc_id == "":
    print("Creating VPC...")
    # Get availability zones
    response = ec2Client.describe_availability_zones(
        Filters=[
            {
                'Name': 'region-name',
                'Values': [region]
            },
            {
                'Name': 'state',
                'Values': ['available']
            },
        ]
    )
    if len(response['AvailabilityZones']) < 2:
        raise SystemExit("Stopping the execution. We need at least 2 availability zones to create private subnets. Make sure the region is set correctly in aws credentials")
    zone1 = response['AvailabilityZones'][0]['ZoneName']
    zone2 = response['AvailabilityZones'][1]['ZoneName']
    # print(zone1, zone2)
    
    #Create VPC

    vpc = ec2.create_vpc(CidrBlock='10.0.0.0/16')
    vpc.wait_until_available()
    vpc_id=vpc.id
    #print(vpc.id)
    
    #Create Private Subnets
    subnet1 = ec2.create_subnet(CidrBlock='10.0.0.0/19', VpcId=vpc_id, AvailabilityZone=zone1)
    subnet1_id=subnet1.id
    #print(subnet1.id)
    
    subnet2 = ec2.create_subnet(CidrBlock='10.0.32.0/19', VpcId=vpc_id,AvailabilityZone=zone2)
    #print(subnet2.id)
    subnet2_id=subnet2.id
    subnets = subnet1_id + "," + subnet2_id

    print("Created VPC {} with private subnets {},{}".format(vpc_id, subnet1_id, subnet2_id))

In [ ]:
# create a new RDS instance
if (create_rds == 'y' or create_rds == 'Y') and len(db_instance_name) > 0:
    print("Creating RDS Instance...")
    #Create RDS DB Subnet
    conn = boto3.client('rds')

    db_subnet_name = db_instance_name+"_dbsubnet"
    print(db_subnet_name)
    response = conn.create_db_subnet_group(
        DBSubnetGroupName=db_subnet_name,
        DBSubnetGroupDescription='rdssubnetgrp',
        SubnetIds=[subnet1_id,subnet2_id],
    )
    
    # Get DB security group id
    response = ec2Client.describe_security_groups(
        Filters=[
            {
                'Name': 'vpc-id',
                'Values': [vpc_id,]
            },
        {
                'Name': 'group-name',
                'Values': ["default"]
            },
        ],
    )
    db_security_group_id=response['SecurityGroups'][0]['GroupId']
    print(db_security_group_id)
    
    # Create RDS
    response = conn.create_db_instance(
            AllocatedStorage=10,
            DBInstanceIdentifier=db_instance_name,
            DBInstanceClass="db.t2.small",
            Engine="mysql",
            MasterUsername="root",
            MasterUserPassword="pass12345",
            Port=3306,
            VpcSecurityGroupIds=[db_security_group_id],
            DBSubnetGroupName = db_subnet_name,
            StorageEncrypted=True
        )
    print(response)
    db_name = response['DBInstance']['DBInstanceIdentifier']
    print("RDS instance created: DB name={}, with credentials (root/pass12345)".format(db_name))

In [ ]:
print("S3 bucket: {} ".format(s3_bucket_name))
print("Region: {}".format(region))
print("Stack name: {}".format(stack_name))
print("Sender email: {}".format(sender_email_id))
print("Recipent email: {}".format(receiver_email_id))
print("SlackWebhooks: {}".format(slack_webhook_urls))
print("isVPC: {}".format(isVpc))
print("VPC: {}".format(vpc_id))
print("Subnets: {}".format(subnets))


### Installing Trapheus with the above parameters

In [ ]:
pip install -r requirements.txt

In [ ]:
%run install.py \
    --s3bucket $s3_bucket_name \
    --region $region \
    --stackname $stack_name \
    --senderemail $sender_email_id \
    --recipientemail $receiver_email_id \
    --slackwebhooks $slack_webhook_urls \
    --isvpc $isVpc \
    --vpcid $vpc_id \
    --subnets $subnets